In [37]:
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm

In [38]:
with open('ClassificationRules.txt', 'r') as file:
    class_rules=[]
    for line in file:
        class_rules.append([float(n) for n in line.strip().split()])
        
class_rules=np.array(class_rules)
print(len(class_rules), len(class_rules[0])) #266 reguł klasyfikacji, każda to ciąg dł. 9350 zawierający liczby ze zbioru klas {1,2,3}

266 9350


In [39]:
with open('ImageExpertReduced.txt', 'r') as file:
    for line in file:
        exemplary=np.array([float(n) for n in line.strip().split()]) #wzorcowa klasyfikacja
print(len(exemplary))

9350


In [40]:
def binary_random(p):
    return (random.uniform(0, 1) < p)

def initial_prob_vector(d):
    return np.array([0.5]*d)

def random_individual(prob_vector,d):
    return np.array([binary_random(prob_vector[i]) for i in range(d)])
    
def random_population(prob_vector,N,d):
    return np.array([random_individual(prob_vector,d) for i in range(N)])

def population_evaluation(P, f, exemplary, class_rules):
    values = [f(i, exemplary, class_rules) for i in P]
    return np.array(values)

def pbil(f,N,T,t1,t2,t3,d, exemplary, class_rules):
    prob_vector = initial_prob_vector(d)
    P = random_population(prob_vector,N,d)
    pop_eval = population_evaluation(P,f, exemplary, class_rules)
    
    for t in tqdm(range(T)):
        best = P[np.argmax(pop_eval)]
        prob_vector = prob_vector*(1-t1)+best*t1
        for k in range(d):
            if random.uniform(0, 1) < t2:
                prob_vector[k]=prob_vector[k]*(1-t3)+binary_random(0.5)*t3
        P = random_population(prob_vector,N,d)
        pop_eval = population_evaluation(P,f, exemplary, class_rules)
        
    return P[np.argmax(pop_eval)]

def classification_quality(chosen_rules, exemplary, class_rules):
    classification = stats.mode(class_rules[chosen_rules], axis=0)[0]
    return np.sum(classification == exemplary)
    
    

In [41]:
N=50
T=1000
t1=0.3
t2=0.3
t3=0.2
d=266
'''
θ1 - współczynnik uczenia
θ2 - prawdopodobieństwo mutacji
θ3 - współczynnik zaburzenia podczas mutacji
'''
result=pbil(classification_quality,N,T,t1,t2,t3,d, exemplary, class_rules)

100%|█████████████████████████████████████| 1000/1000 [2:50:51<00:00, 10.25s/it]


In [42]:
classification_quality(result, exemplary, class_rules)

9146

In [43]:
print(classification_quality(result, exemplary, class_rules)/9350)

0.9781818181818182
